# OntoGraph Usage - Jupyter Notebook

[//]: # (------------------------------------------    DO NOT MODIFY THIS    ------------------------------------------)
<style type="text/css">
.tg  {border-collapse:collapse;
      border-spacing:0;
     }
.tg td{border-color:black;
       border-style:solid;
       border-width:1px;
       font-family:Arial, sans-serif;
       font-size:14px;
       overflow:hidden;
       padding:10px 5px;
       word-break:normal;
      }
.tg th{border-color:black;
       border-style:solid;
       border-width:1px;
       font-family:Arial, sans-serif;
       font-size:14px;
       font-weight:normal;
       overflow:hidden;
       padding:10px 5px;
       word-break:normal;
      }
.tg .tg-fymr{border-color:inherit;
             font-weight:bold;
             text-align:left;
             vertical-align:top
            }
.tg .tg-0pky{border-color:inherit;
             text-align:left;
             vertical-align:top
            }
[//]: # (--------------------------------------------------------------------------------------------------------------)
[//]: # (-------------------------------------    FILL THIS OUT WITH YOUR DATA    -------------------------------------)
[//]: # (--------------------------------------------------------------------------------------------------------------)
</style>
<table class="tg">
    <tbody>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Title:</td>
        <td class="tg-0pky">OntoGraph Usage - Jupyter Notebook </td>
      </tr>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Authors:</td>
        <td class="tg-0pky">
            <a href="https://github.com/ecarrenolozano" target="_blank" rel="noopener noreferrer">Edwin Carreño</a>,
            <a href="" target="_blank" rel="noopener noreferrer">Denes Türei</a>
        </td>
      </tr>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Affiliations:</td>
        <td class="tg-0pky">
            <a href="https://www.ssc.uni-heidelberg.de/en" target="_blank" rel="noopener noreferrer">Scientific Software Center</a>,
            <a href="https://saezlab.org/" target="_blank" rel="noopener noreferrer">Saez-Rodriguez Group</a>
        </td>
      </tr>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Date Created:</td>
        <td class="tg-0pky">30.10.2024</td>
      </tr>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Description:</td>
        <td class="tg-0pky">This notebook is a sandbox to explore, test, and demonstrate the core functionalities of the `OntoGraph` package, helping refine the user experience and gather early feedback before writing the official documentation.</td>
      </tr>
    </tbody>
</table>

[//]: # (--------------------------------------------------------------------------------------------------------------)

## Overview

This notebook serves as a **sandbox environment** to explore and test the functionalities of the `OntoGraph` Python package. It is designed to:  

- Evaluate the **user experience** during development.  
- Demonstrate common use cases and workflows to potential users.  
- Collect feedback from collaborators before finalizing the official documentation.
- Help to create use cases based on what the user wants.

Feel free to execute, modify, and extend the examples in this notebook to better understand how the package can be used in real scenarios.

## How to Use This Notebook

1. Run cells sequentially from top to bottom.
2. If using an environment manager (e.g., `conda`, `venv`), activate it before running.
3. To reset, click `Kernel -> Restart & Run All`.

## Importing Libraries

In [ ]:
# 1. Standard library imports
import sys

sys.path.append('/home/ecarreno/SSC-Projects/b_REPOSITORIES/ontograph')

from pathlib import Path

# 2. Related third party imports
# 3. Local application/library specific imports
from ontograph.downloader import PoochDownloaderAdapter
from ontograph.ontology_query import OntologyQueries
from ontograph.ontology_loader import ProntoLoaderAdapter
from ontograph.ontology_registry import OBORegistryAdapter

## Defining a `cache` folder

In this cache folder we are going to store all the ontologies and other files like the ontology registry metadata from OBO foundry.

In [ ]:
cache_dir = Path('../data/out')

## Step 1. Create a registry with all the ontologies

We are going to create a registry based on all the metadata that OBO Foundy has.

In [ ]:
onto_registry = OBORegistryAdapter(cache_dir=cache_dir)

### Load the registry (in case of not having the registry it will be downloaded automatically)

In [ ]:
onto_registry.load_registry()

### Print registry' schema

In [ ]:
onto_registry.print_registry_schema_tree()

### List of available ontologies

In [ ]:
for ontology in onto_registry.list_available_ontologies():
    print(ontology)

### Print the link associated to a valid ontology (e.g., 'chebi')

In [ ]:
# This function looks if the given ontology has a link with a given extension

print('Link: {}'.format(onto_registry.get_download_url('chebi', 'obo')))

### Print available formats for a valid ontology

In [ ]:
print(onto_registry.get_available_formats(ontology_id='chebi'))

## Step 2. Download specific ontologies

### Create a downloader object

In [ ]:
downloader = PoochDownloaderAdapter(
    cache_dir=cache_dir,
    registry=onto_registry,
)

### Download multiple ontologies at once

In [ ]:
# Put the ontologies and their formats as a dictionary
resources = [
    {'name_id': 'chebi', 'format': 'owl'},
    {'name_id': 'go', 'format': 'obo'},
    {'name_id': 'ado', 'format': 'owl'},
]

# Use the downloader, it has a cache in case you have already downloaded the ontologies.
batch_results = downloader.fetch_batch(resources)

# Print all the paths where your ontologies are located
print('Ontologies in cache:')
for ontology_name, ontology_path in batch_results.items():
    print(f'\t{ontology_name}: {ontology_path}')

## Step 3. Load ontologies

### Create a loader for the ontology. Behind scenes it uses `Pronto`. However, it can be another one. You as a final user will not notice this.

In [ ]:
ontology_loader = ProntoLoaderAdapter(cache_dir=cache_dir)

### Load the `go` ontology in `obo` format

In [ ]:
name_id_go = 'go'
format_go = 'obo'

gene_ontology = ontology_loader.load(name_id=name_id_go, format=format_go)

print(f'Loaded ontology: {name_id_go}.{format_go}')
print(f'Number of terms: {len(gene_ontology.terms())}')

### Step 4. Query the ontology

In [ ]:
# term_id = "GO:0008150" # biological_process
# term_id = "GO:0160266"  # anestrus phase
term_id = 'GO:0070360'  # symbiont-mediated actin polymerization-dependent cell-to-cell migration in host

In [ ]:
queries = OntologyQueries(gene_ontology)

# Print term relations
print(f'Term: {term_id}')
print(f'  Parents     : {queries.parents(term_id)}')
print(f'  Children    : {queries.children(term_id)}')


# print(f"  Ancestors   : {queries.ancestors(term_id)}")
# print(f"  Descendants : {queries.descendants(term_id)}")

## References and Further Reading

- [OBO Foundry](https://obofoundry.org/)
- [`pronto` documentation](https://pronto.readthedocs.io/en/stable/)
- [`pooch` documentation](https://www.fatiando.org/pooch/latest/)